In [13]:
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point

In [14]:
# Ruta del archivo CSV
ruta_csv = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Recorridos vehiculos\2025-03_ALEJANDRO+CHOFER_History+tracking+report(20250410).csv'
# Cargar el CSV
df_recorrido_vehiculo = pd.read_csv(ruta_csv, encoding='latin1')
# Mostrar las primeras filas
df_recorrido_vehiculo

,No.,Device ID,Locate time,Speed,Status,Longitude,Latitude,Location
0,1,ALEJANDRO CHOFER,"\t""2025-03-01 00:00:05""",91,Direction:114;Locate type:GPS;,-60.264804,-17.993084,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S..."
1,2,ALEJANDRO CHOFER,"\t""2025-03-01 00:00:25""",74,Direction:100;Locate type:GPS;,-60.260516,-17.994336,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S..."
2,3,ALEJANDRO CHOFER,"\t""2025-03-01 00:02:26""",89,Direction:94;Locate type:GPS;,-60.236524,-17.996451,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S..."
3,4,ALEJANDRO CHOFER,"\t""2025-03-01 00:03:25""",85,Direction:133;Locate type:GPS;,-60.224844,-18.001802,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S..."
4,5,ALEJANDRO CHOFER,"\t""2025-03-01 00:03:45""",88,Direction:133;Locate type:GPS;,-60.221604,-18.004709,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S..."
...,...,...,...,...,...,...,...,...
20443,20444,ALEJANDRO CHOFER,"\t""2025-03-29 18:30:41""",0,Direction:0;ACC status:Off;Locate type:GPS;Bat...,-63.259280,-17.343500,"Calle Sucre, Montero, Provincia Obispo Santist..."
20444,20445,ALEJANDRO CHOFER,"\t""2025-03-29 19:29:48""",0,Direction:0;ACC status:Off;Locate type:GPS;Bat...,-63.259280,-17.343500,"Calle Sucre, Montero, Provincia Obispo Santist..."
20445,20446,ALEJANDRO CHOFER,"\t""2025-03-29 20:38:35""",0,Direction:0;ACC status:Off;Locate type:LBS;Bat...,-63.252079,-17.354738,"Calle Leonardo Arteaga, Montero, Provincia Obi..."
20446,20447,ALEJANDRO CHOFER,"\t""2025-03-29 20:38:37""",0,Direction:0;ACC status:Off;Locate type:GPS;Bat...,-63.259084,-17.343842,"Calle Sucre, Montero, Provincia Obispo Santist..."


In [15]:
# Crear geometría de puntos a partir de las columnas de longitud y latitud
geometry = [Point(xy) for xy in zip(df_recorrido_vehiculo['Longitude'], df_recorrido_vehiculo['Latitude'])]

# Crear el GeoDataFrame con sistema de coordenadas WGS84 (EPSG:4326)
gdf = gpd.GeoDataFrame(df_recorrido_vehiculo, geometry=geometry, crs='EPSG:4326')

# Reproyectar a UTM zona 20S (EPSG:32720)
gdf_utm = gdf.to_crs(epsg=32720)

# Mostrar resultado
gdf_utm.head()

,No.,Device ID,Locate time,Speed,Status,Longitude,Latitude,Location,geometry
0,1,ALEJANDRO CHOFER,"\t""2025-03-01 00:00:05""",91,Direction:114;Locate type:GPS;,-60.264804,-17.993084,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S...",POINT (789655.888 8008442.718)
1,2,ALEJANDRO CHOFER,"\t""2025-03-01 00:00:25""",74,Direction:100;Locate type:GPS;,-60.260516,-17.994336,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S...",POINT (790108.312 8008297.458)
2,3,ALEJANDRO CHOFER,"\t""2025-03-01 00:02:26""",89,Direction:94;Locate type:GPS;,-60.236524,-17.996451,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S...",POINT (792647.050 8008025.436)
3,4,ALEJANDRO CHOFER,"\t""2025-03-01 00:03:25""",85,Direction:133;Locate type:GPS;,-60.224844,-18.001802,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S...",POINT (793875.851 8007414.347)
4,5,ALEJANDRO CHOFER,"\t""2025-03-01 00:03:45""",88,Direction:133;Locate type:GPS;,-60.221604,-18.004709,"RN4: San Jos¨¦ de Chiquitos-Robor¨¦, Ipi¨¢s, S...",POINT (794214.343 8007087.310)
